# CSCI 3202, Spring 2022: Final Coding Project

---


This is your final programming project for CSCI 3202. It is due on Canvas by **11:59 PM on Saturday April 30**.  Your solutions to theoretical questions should be done in Markdown/LateX directly below the associated question. Your solutions to computational questions should include any relevant Python code, as well as results and any written commentary.

You have two options for completing your final project for this course. The first option is presented in this notebook and involves implementing a reinforcement learning algorithm and producing a five-minute video that explains your process of solving this problem. The second option is to design your own project that includes the algorithms we've discussed since the midterm - Bayes Nets, Hidden Markov Models, Markov Decision Processes, or Reinforcement Learning - or an algorithm related to one of these that we haven't discussed in class. Your project also needs to include some kind of analysis of how it performed on a specific problem. If you're interested in the design your own project option, you need to discuss your idea with one of the course instructors to get approval. If you do a project without getting approval, you will receive a 0 regardless of the quality of the project. You will also need to produce a short, five-minute video that explains your project.

**The rules:**

1. Choose EITHER the given problem to submit OR choose your own project topic. 

2. If you choose your own project topic, please adhere to the following guidelines:
- The project needs to be approved by the course instructors.
- The project needs to include one of the algorithms we've discussed since the midterm - Bayes Nets, HMMs, MDPs, or Reinforcement Learning - or an algorithm that we haven't discussed in class. 
- If you do your own project without prior approval, you will receive a 0 for this project.
- Your project code, explanation, and results must all be contained in a Jupyter notebook. 

3. All work, code and analysis must be **your own**.
4. You may use your course notes, posted lecture slides, textbook, in-class notebooks and homework solutions as resources.  You may also search online for answers to general knowledge questions, like the form of a probability distribution function, or how to perform a particular operation in Python. You may not use entire segments of code as solutions to any part of this project, e.g. if you find a Python implementation of policy iteration online, you can't use it.
5. You may **not** post to message boards or other online resources asking for help.
6. **You may not collaborate with classmates or anyone else.**
7. This is meant to be like a coding portion of your final exam. So, we will be much less helpful than we typically am with homework. For example, we will not check answers, help debug your code, and so on.
8. If you have a question, post it first as a **private** Piazza message. If we decide that it is appropriate for the entire class, then we will make it a public post (and anonymous).
9. If something is left open-ended, it is probably because we intend for you to code it up however you want, and only care about the plots/analysis we see at the end. Feel free to ask clarifying questions though.

Violation of these rules will result in an **F** and a trip to the Honor Code council.

---
**By writing your name below, you agree to abide by these rules:**

**Your name:**

---



Chapter 22 in textbook
Lecture schedule for reinforcement learnng. Sutton text book
Q-Learning

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict

# added packages
import heapq
from matplotlib import colors



---
## [100 pts] Problem 1:  Reinforcement learning

Consider a **cube** state space defined by $0 \le x, y, z \le L$. Suppose you are piloting/programming a drone to learn how to land on a platform at the center of the $z=0$ surface (the bottom). Some assumptions:
* In this discrete world, if the drone is at $(x,y,z)$ it means that the box is centered at $(x,y,z)$. There are boxes (states) centered at $(x,y,z)$ for all $0 \le x,y,z \le L$. Each state is a 1 unit cube. So when $L=2$ (for example), there are cubes centered at each $x=0,1,2$, $y=0,1,2$ and so on, for a total state space size of $3^3 = 27$ states.
* All of the states with $z=0$ are terminal states.
* The state at the center of the bottom of the cubic state space is the landing pad. For example, when $L=4$, the landing pad is at $(x,y,z) = (2,2,0)$.
* All terminal states ***except*** the landing pad have a reward of -1. The landing pad has a reward of +1.
* All non-terminal states have a living reward of -0.01.
* The drone takes up exactly 1 cubic unit, and begins in a random non-terminal state.
* The available actions in non-terminal states include moving exactly 1 unit Up (+z), Down (-z), North (+y), South (-y), East (+x) or West (-x). In a terminal state, the training episode should end.

#### Part A
Write a class `MDPLanding` to represent the Markov decision process for this drone. Include methods for:
1. `actions(state)`, which should return a list of all actions available from the given state
2. `reward(state)`, which should return the reward for the given state
3. `result(state, action)`, which should return the resulting state of doing the given action in the given state

and attributes for:
1. `states`, a list of all the states in the state space, where each state is represented as an $(x,y,z)$ tuple
2. `terminal_states`, a dictionary where keys are the terminal state tuples and the values are the rewards associated with those terminal states
3. `default_reward`, a scalar for the reward associated with non-terminal states
4. `all_actions`, a list of all possible actions (Up, Down, North, South, East, West)
5. `discount`, the discount factor (use $\gamma = 0.999$ for this entire problem)

How you feed arguments/information into the class constructor is up to you.

Note that actions are *deterministic* here.  The drone does not need to learn transition probabilities for outcomes of particular actions. What the drone does need to learn, however, is where the landing pad is, and how to get there from any initial state.

Before moving on to Part B, we recommend that you test that your MDPLanding code is set up correctly. Write unit tests that display the actions for a given state, rewards, results, etc. This will help you identify errors in your implementation and save you a lot of debugging time later.

In [36]:
class MDPLanding:
    def __init__(self, len):
        self.states = [(x, y, z) for x in range(0, len) for y in range(0, len) for z in range(0, len)]
        self.terminal_states = {}
        # All states where z = 0
        for state in self.states:
            if state[2] == 0:
                self.terminal_states[state] = -1
        self.terminal_states[(int(len/2),int(len/2),0)] = 1
        self.default_reward = -0.01
        self.all_actions = [(0,1,0), (0,-1,0), (-1,0,0), (1,0,0), (0,0,-1), (0,0,1)]
        self.discount = 0.999
        self.len = len

    def actions(self, state):
        acts = []
        for move in self.all_actions:
            n = (state[0] + move[0], state[1] + move[1], state[2] + move[2])
            if n[0] < 0 or n[0] >= self.len:
                continue
            elif n[1] < 0 or n[1] >= self.len:
                continue
            elif n[2] < 0 or n[2] >= self.len:
                continue
            else:
                acts.append(move)
        return acts

    def reward(self, state):
        if state in self.terminal_states:
            return self.terminal_states[state]
        else:
            return self.default_reward

    def result(self, state, action):
        return (state[0] + action[0], state[1] + action[1], state[2] + action[2])


test = MDPLanding(3)

# for state in test.states:
#     print(state)
# print(len(test.states))
actions = test.actions((2,2,2))
print(actions)
print(len(actions))
print()
actions = test.actions((1,1,1))
print(actions)
print(len(actions))
print()
test = MDPLanding(2)
print(len(test.states))
test = MDPLanding(3)
print(len(test.states))
test = MDPLanding(4)
print(len(test.states))
test = MDPLanding(5)
print(len(test.states))

test = MDPLanding(4)
print(test.actions((0, 3, 3)))


[(0, -1, 0), (-1, 0, 0), (0, 0, -1)]
3

[(0, 1, 0), (0, -1, 0), (-1, 0, 0), (1, 0, 0), (0, 0, -1), (0, 0, 1)]
6

8
27
64
125
[(0, -1, 0), (1, 0, 0), (0, 0, -1)]


#### Part B
Write a function to implement **policy iteration** for this drone landing MDP. Create an MDP environment to represent the $L=4$ case (so 125 total states).

Use your function to find an optimal policy for your new MDP environment. Check (by printing to screen) that the policy for the following states are what you expect, and **comment on the results**:
1. $(2,2,1)$
1. $(0,2,1)$
1. $(2,0,1)$

The policy for each of these states is the action that the agent should take in that state. 

In [51]:
mdp = MDPLanding(4)

def find_policies(mdp):
    # Set the default policy to move down
    policies = {state : (0, -1, 0) for state in mdp.states}
    utilities = {state : 0 for state in mdp.states}

    # Loop until the change is marginal
    #for _ in range(0,100):
    while True:
        #print('Starting iter: ', count)
        delta = 0.0
        # For each state in the graph, find the best policy
        for state in mdp.states:
            # Get all possible moves for the given state
            actions = mdp.actions(state)
            # Need to track max utility
            max = (-np.infty, (0, 0, 0))

            if len(actions) == 0:
                print('No actions?!: ', state)
            if state in mdp.terminal_states:
                #print('Terminal')

                # util = mdp.reward(state)
                # if util != utilities[state]:
                #     diff = np.abs(util - utilities[state])
                #     if diff > delta:
                #         delta = diff

                policies[state] = (0,0,0)
                utilities[state] = mdp.reward(state)
                continue

            # For each action the state can undertake, determine the new utility of the move
            # and find the best one
            for i,act in enumerate(actions):
                # Calculate the utility of the move
                util = mdp.discount * mdp.reward(mdp.result(state, act)) + utilities[mdp.result(state, act)]
                #util += utilities[state]

                # If the utility is better than the previous best, track this one
                if util > max[0]:
                    max = (util, act)
            # Determine how much the policy change altered the utility
            diff = np.abs(utilities[state] - max[0])
            # If this is the new highest change, update delta
            if diff > delta:
                delta = diff
            # Update the policy and utility
            utilities[state] = max[0]
            policies[state] = max[1]
        # If only a marginal change, exit
        if delta < 0.001:
            break
        # else:
        #     print(delta)

    #print(policies)
    return policies, utilities

policies, utilities = find_policies(mdp)

print('(2,2,1): ', policies[(2,2,1)])
print('(0,2,1): ', policies[(0,2,1)])
print('(2,0,1): ', policies[(2,0,1)])
# print()
# print('(2,2,1): ', utilities[(2,2,1)])
# print('(0,2,1): ', utilities[(0,2,1)])
# print('(2,0,1): ', utilities[(2,0,1)])
# for pol in policies:
#     print(pol, ' ', policies[pol], ' ', utilities[pol])

(2,2,1):  (0, 0, -1)
(0,2,1):  (1, 0, 0)
(2,0,1):  (0, 1, 0)


- State: (2,2,1) Action: (0, 0, -1)
> This action makes sense for this state because this state is directly above the landing spot. Therefore, the best action to take is to move down.
- State: (0,2,1) Action: (1,0,0)
> This action makes sense for this state because if the drone moves directly down, it hits an undesired terminal state. If it moves more central, the drone moves closer to the desired terminal state and therefore has more utility.
3. State: (2,0,1) Action: (0, 1, 0)
> Similar to number 2, the drone is right above a negative terminal state. Therefore, the action with the most utility is to move more central and get closer to the desired terminal state.

#### Part C

Code up a **Q-learning** agent/algorithm to learn how to land the drone. You can do this however you like, as long as you use the MDP class structure defined above.  

Your code should include some kind of a wrapper to run many trials to train the agent and learn the Q values.  You also do not need to have a separate function for the actual "agent"; your code can just be a "for" loop within which you are refining your estimate of the Q values.

From each training trial, save the cumulative discounted reward (utility) over the course of that episode. That is, add up all of $\gamma^t R(s_t)$ where the drone is in state $s_t$ during time step $t$, for the entire sequence. We refer to this as "cumulative reward" because we usually refer to "utility" as the utility *under an optimal policy*.

Some guidelines:
* The drone should initialize in a random non-terminal state for each new training episode.
* The training episodes should be limited to 50 time steps, even if the drone has not yet landed. If the drone lands (in a terminal state), the training episode is over.
* You may use whatever learning rate $\alpha$ you decide is appropriate, and gives good results.
* There are many forms of Q-learning. You can use whatever you would like, subject to the reliability targets in Part D below.
* Your code should return:
  * The learned Q values associated with each state-action pair.
  * The cumulative reward for each training trial. 
  * Anything else that might be useful in the ensuing analysis.

In [ ]:
# Your code here.
# https://www.simplilearn.com/tutorials/machine-learning-tutorial/what-is-q-learning
# https://gibberblot.github.io/rl-notes/single-agent/function-approximation.html

#### Part D

Initialize the $L=10$ environment (so that the landing pad is at $(5,5,0)$). Run some number of training trials to train the drone.

**How do I know if my drone is learned enough?**  If you take the mean cumulative reward across the last 5000 training trials, it should be around 0.80. This means at least about 10,000 (but probably more) training episodes will be necessary. It will take a few seconds on your computer, so start small to test your code.

**Then:** Compute block means of cumulative reward from all of your training trials. Use blocks of 500 training trials. This means you need to create some kind of array-like structure such that its first element is the mean of the first 500 trials' cumulative rewards; its second element is the mean of the 501-1000th trials' cumulative rewards; and so on. Make a plot of the block mean rewards as the training progresses. It should increase from about -0.5 initially to somewhere around +0.8.

**And:** Print to the screen the mean of the last 5000 trials' cumulative rewards, to verify that it is indeed about 0.80.

In [ ]:
# Your code here.

#### Part E

**Question 1:** Why does the cumulative reward start off around -0.5 at the beginning of the training?

**Question 2:** Why will it be difficult for us to train the drone to reliably obtain rewards much greater than about 0.8?

#### Part F
Choose three other reward structures, including the terminal rewards and the living rewards, and rerun your policy iteration and Q-learning algorithms with those reward structures. Write a paragraph or two describing the reward structures that you selected, what impact you expected the changes to have on your results, and what changes actually happened. If there was a difference in what you expected and what actually happened, reflect on why there was a difference. 